In [ ]:
!pip install tf-nightly

In [ ]:
import tensorflow as tf
tf.__version__

'2.6.0-dev20210529'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/projects/mai_hdr

/content/drive/MyDrive/projects/ntire21/mai_hdr/a_dw_mutone_crloss


In [ ]:
import tensorflow as tf
from model import get_model

In [ ]:
model = get_model([460, 720, 3])
ckpt = tf.train.Checkpoint(model = model)
ckpt_dir = 'train_ckpts/xxxx-xxxx/last'

In [ ]:
chkpt_manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=1)
ckpt.restore(chkpt_manager.latest_checkpoint)
_ = model(tf.ones([1,460,720,3])) # dummy input to build the graph  #(1, 1060, 1900, 3)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmpd_7vwlpg/assets


In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/projects/mai_hdr/model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)